In [16]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

In [17]:
def im_multi(path):
    #Input: file path of image
    #Output: [path, {'size': size of image at path}]
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

In [18]:
def im_stats(im_stats_df): 
    #Input: dataframe of training images (type, image, path)
    #Output: dataframe of training images (type, image, path, size)
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    #p.map(f, [x, y, z]) returns a list [f[x], f[y], f[z]]
    #For all paths in the inputted dataframe im_stats_df, they are passed through im_multi
    #ret holds [[path, {'size': size of image at path}], ...]
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in 
                                                                     im_stats_d[x]['size']))
    #Adds additional column to original dataframe and formats size as 3264 4160
    return im_stats_df

In [19]:
def get_im_cv2(path):
    #Input: file path of image
    #Output: [original path, resized image]
    img = cv2.imread(path)
    resized = cv2.resize(img, (64, 64), cv2.INTER_LINEAR) #INTER_LINEAR is algorithm 
    #to downsize image
    #I could try using INTER_AREA as, according to the URL below, could be better
    #http://tanbakuchi.com/posts/comparison-of-openv-interpolation-algorithms/
    return [path, resized]

In [20]:
def normalize_image_features(paths):
    #Input: list of paths
    #Output: list of resized images that have been transposed for Conv2d layer
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    #ret holds a list: [[image path, resized image], ...]
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
        #imf_d[image path in ret] = resized image
    ret = []
    fdata = [imf_d[f] for f in paths]
    #fdata holds a list: [resized image, ...]
    fdata = np.array(fdata, dtype=np.uint8)
    #fdata is now a numbpy array of ints
    fdata = fdata.transpose((0, 3, 1, 2))
    #Usually its (2, 0, 1) since it changes the image from (0, 1, 2)->(width, height, channel)
    #to (2, 0, 1)->(channel, width, height) for the Conv2d layer, 
    #but since we have 4 dimensions, it gets bumped up one to (3, 1, 2);
    #I don't get why it has 4 dimensions tho (https://skymind.ai/wiki/convolutional-network)
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    #fdata now has values between 0 and 1
    return fdata

In [21]:
train = glob.glob("/Users/keerat/dev/AOSResearch/resources/Train/**/*.jpg")
#train is an array holding all of the path files in the training set
print("Number of files in training set: %", len(train))
train = pd.DataFrame([[p.split('/')[7],p.split('/')[8],p] for p in train], columns = 
                     ['type','image','path'])[::1]
#train is a dataframe holding the type (ex. "Type_1"), image name (ex. "0.jpg"), and file path
#(ex. "/Users/keerat/dev/AOSResearch/resources/Train/Type_1/0.jpg")
train = im_stats(train)
#train now has additional column with size (ex. 3264 4160)
train = train[train['size'] != '0 0'].reset_index(drop=True) #corrupt images removed
#train now has an additional column with the index reset to 0, 1, 2, 3... instead of 0, 5, 10..
print("loading train data")
train_data = normalize_image_features(train['path'])
#train_data holds a usable set of training images for the CNN
print("train data loaded")
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)
#train.npy is a file that has all of the image arrays in train_data

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values) 
#train_target holds type of each image in train
#Type_1 = 0, Type_2 = 1, Type_3 = 2
#For example, if the 40th image in train is Type_1, then train_target[40] = 0
print(le.classes_)  
np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)
#train_target.npy is a file that has all values of train_target

test = glob.glob("/Users/keerat/dev/AOSResearch/resources/test/*.jpg")
#test is an array holding all of the path files in the test set
test = pd.DataFrame([[p.split('/')[7],p] for p in test], columns = ['image','path']) [::1]
#test is a dataframe holding the image name (ex. "0.jpg"), and file path
#(ex. "/Users/keerat/dev/AOSResearch/resources/test/0.jpg")
print("loading test data")
test_data = normalize_image_features(test['path'])
#test_data holds a usable set of test images for the CNN
np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)
#test.npy is a file that has all of the image arrays in test_data
print("test data loaded")
test_id = test.image.values
np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)
#test_id.npy is a file that has all of the image names (ex. '0.jpg) in test_data

Number of files in training set: % 1478
loading train data
train data loaded
['Type_1' 'Type_2' 'Type_3']
loading test data
test data loaded


In [22]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
K.set_image_dim_ordering('th')
K.set_floatx('float32')
np.random.seed(17)

In [23]:
train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

In [24]:
def create_model(opt_='adamax'):
    model = Sequential()
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th', 
                            input_shape=(3, 64, 64))) 
    #Could try different input shape
    
    #Activation='relu' to discover nonlinear patterns of data
    #dim_ordering = 'th' to match (0, 3, 1, 2) images were transposed to
    #Four 3x3 filters
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Dropout(0.2))
    #Sets a fraction of rate of input units to 0 to prevent overfitting
    model.add(Flatten())
    #Creates 1D feature vector for Dense layers
    model.add(Dense(12, activation='tanh')) #Classifies
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='softmax')) #Classifies

    model.compile(optimizer=opt_, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #Compiles all layers of model with optimizer, loss function, and metrics (to evaluate 
    #performance)
    return model

In [25]:
def cleanImages():
    datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
    #rotation_range = random rotation of images up to 0.3 degrees
    #zoom_range = random zoom of images up to a scale of 0.3
    datagen.fit(train_data)
    return datagen

In [26]:
def fitAndPredict(): #Runs data through model
    print("cleaning images")
    datagen=cleanImages() #datagen now points to the parameters in cleanImages()
    print("images cleaned")
    
    model = create_model() #model holds CNN model
    x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,
                                                               test_size=0.4, random_state=17)
    #x_train = training set of images (60% of original training set)
    #x_val_train = validation set of images (40 % of original training set)
    #y_train = types for training images
    #y_val_train = types for validation images
    
    #Training set is used to fit the parameters using back prop
    #Validation set is used to fine tune parameters to create a final model
    #Test set is used to assess model's performance
    print("fitting data")
    model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), 
                        nb_epoch=200, samples_per_epoch=len(x_train), 
                        verbose=2, validation_data=(x_val_train, y_val_train))
    #Training set is augmented real-time with datagen.flow
    #CNN processes images not one at a time, but in batches.  With batch_size = 15, one batch 
    #is 15 x 3 x 64 x 64.  The batch can't be too big, or else the machine can't handle it,
    #and it can't be too small or else there will be no variance within the batch.
    print("data fitted in model")
    test_data = np.load('test.npy')
    test_id = np.load('test_id.npy')
    print("creating predictions")
    predictions = model.predict_proba(test_data)
    #Runs test_data through model and returns probablibity of it being each type
    print("predictions made")
    return predictions

In [27]:
def test(isTrue): #Runs CNN on test images
    pred=fitAndPredict()
    print("creating test file")
    df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3']) #Instantiates dataframe
    df['image_name'] = test_id #image_name holds the .jpg file name
    if (isTrue): #if(True), it will create a .csv file with the dataframe
        df.to_csv('test.csv', index=False)
        print("Test file created in users/keerat/...")
    else: #if(False), it will just show the dataframe
        print(df.to_string())

In [28]:
if __name__ == '__main__':
    
    test(isTrue = False)

cleaning images
images cleaned


/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", input_shape=(3, 64, 64..., data_format="channels_first")`
  after removing the cwd from sys.path.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", data_format="channels_first")`
  # This is added back by InteractiveShellApp.init_path()
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), d

fitting data


/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=2, validation_data=(array([[[..., steps_per_epoch=59, epochs=200)`


Epoch 1/200
 - 7s - loss: 0.9973 - acc: 0.5355 - val_loss: 1.0534 - val_acc: 0.5203
Epoch 2/200
 - 6s - loss: 1.0070 - acc: 0.5233 - val_loss: 1.0248 - val_acc: 0.5203
Epoch 3/200
 - 6s - loss: 0.9879 - acc: 0.5323 - val_loss: 1.0350 - val_acc: 0.5203
Epoch 4/200
 - 6s - loss: 0.9949 - acc: 0.5109 - val_loss: 1.0272 - val_acc: 0.5203
Epoch 5/200
 - 6s - loss: 0.9713 - acc: 0.5411 - val_loss: 1.0236 - val_acc: 0.5203
Epoch 6/200
 - 6s - loss: 0.9738 - acc: 0.5423 - val_loss: 1.0503 - val_acc: 0.5203
Epoch 7/200
 - 7s - loss: 0.9639 - acc: 0.5346 - val_loss: 1.0257 - val_acc: 0.5118
Epoch 8/200
 - 6s - loss: 0.9748 - acc: 0.5276 - val_loss: 1.0802 - val_acc: 0.5203
Epoch 9/200
 - 6s - loss: 0.9778 - acc: 0.5323 - val_loss: 1.0459 - val_acc: 0.5203
Epoch 10/200
 - 6s - loss: 0.9621 - acc: 0.5558 - val_loss: 1.0295 - val_acc: 0.5051
Epoch 11/200
 - 6s - loss: 0.9589 - acc: 0.5456 - val_loss: 1.0386 - val_acc: 0.5101
Epoch 12/200
 - 7s - loss: 0.9615 - acc: 0.5435 - val_loss: 1.0253 - val_a

Epoch 98/200
 - 6s - loss: 0.7640 - acc: 0.6688 - val_loss: 0.9787 - val_acc: 0.5338
Epoch 99/200
 - 6s - loss: 0.7391 - acc: 0.6668 - val_loss: 0.9876 - val_acc: 0.5355
Epoch 100/200
 - 6s - loss: 0.7317 - acc: 0.6803 - val_loss: 0.9744 - val_acc: 0.5372
Epoch 101/200
 - 7s - loss: 0.6973 - acc: 0.6748 - val_loss: 0.9930 - val_acc: 0.5338
Epoch 102/200
 - 6s - loss: 0.7534 - acc: 0.6576 - val_loss: 0.9796 - val_acc: 0.5507
Epoch 103/200
 - 6s - loss: 0.7888 - acc: 0.6442 - val_loss: 0.9745 - val_acc: 0.5456
Epoch 104/200
 - 7s - loss: 0.7112 - acc: 0.6995 - val_loss: 0.9768 - val_acc: 0.5557
Epoch 105/200
 - 8s - loss: 0.7224 - acc: 0.6938 - val_loss: 0.9691 - val_acc: 0.5389
Epoch 106/200
 - 6s - loss: 0.7342 - acc: 0.6711 - val_loss: 1.0051 - val_acc: 0.5473
Epoch 107/200
 - 6s - loss: 0.7057 - acc: 0.7107 - val_loss: 0.9814 - val_acc: 0.5372
Epoch 108/200
 - 6s - loss: 0.6983 - acc: 0.6892 - val_loss: 0.9856 - val_acc: 0.5389
Epoch 109/200
 - 6s - loss: 0.7305 - acc: 0.6755 - val_l

Epoch 194/200
 - 8s - loss: 0.6151 - acc: 0.7367 - val_loss: 1.0709 - val_acc: 0.5456
Epoch 195/200
 - 11s - loss: 0.6115 - acc: 0.7400 - val_loss: 1.0589 - val_acc: 0.5422
Epoch 196/200
 - 10s - loss: 0.6186 - acc: 0.7245 - val_loss: 1.0444 - val_acc: 0.5270
Epoch 197/200
 - 8s - loss: 0.6442 - acc: 0.7098 - val_loss: 1.0729 - val_acc: 0.5355
Epoch 198/200
 - 8s - loss: 0.6701 - acc: 0.7234 - val_loss: 1.0591 - val_acc: 0.5304
Epoch 199/200
 - 8s - loss: 0.6262 - acc: 0.7197 - val_loss: 1.0913 - val_acc: 0.5541
Epoch 200/200
 - 8s - loss: 0.6581 - acc: 0.7121 - val_loss: 1.0539 - val_acc: 0.5270
data fitted in model
creating predictions
predictions made
creating test file
       Type_1    Type_2    Type_3 image_name
0    0.131986  0.450802  0.417212      0.jpg
1    0.008711  0.029944  0.961346      1.jpg
2    0.048231  0.136938  0.814831     10.jpg
3    0.037731  0.560056  0.402213    100.jpg
4    0.246215  0.055139  0.698646    101.jpg
5    0.060885  0.055761  0.883354    102.jpg
6  